<h1>Chapter 13. Interfaces, Protocols and ABC</h1>

Object-oriented programming is about interfaces. If you want to understand what a type does in Python, find out what methods it provides (its interface).

<h2>Typing Types</h2>

<p><b>Duck Typing</b>:<br>
&emsp;&emsp;Definition: A programming style where an object's suitability is determined by the presence of certain methods and properties rather than the object's actual type.<br>
&emsp;&emsp;Example: If an object has a <code>quack</code> method, it can be treated as a "duck" even if it doesn't inherit from a specific duck class.</p>
<p><b>Goose Typing</b>:<br>
&emsp;&emsp;Definition: A variation of duck typing that also considers the types of the object's methods and attributes, ensuring they match expected types.<br>
&emsp;&emsp;Example: An object not only needs to have certain methods and attributes, but they must also adhere to specific types, e.g., method <code>quack</code> must take and return specific types.</p>
<p><b>Static Typing</b>:<br>
&emsp;&emsp;Definition: A programming style where variable types are explicitly declared and determined at compile-time.<br>
&emsp;&emsp;Example: In languages like Java, you must declare a variable's type (e.g., `int x = 5;`), and it cannot change.</p>
<p><b>Static Duck Typing</b>:<br>
&emsp;&emsp;Definition: A hybrid approach that combines static typing with duck typing.<br>
&emsp;&emsp;Example: A function can specify that it expects an object that has a particular method signature, allowing type-checking at compile time while maintaining flexibility in what kinds of objects can be passed in.</p>

<h2>Two Types of Protocols</h2>

<p><b>Dynamic Protocol</b>:<br>
&emsp;&emsp;Definition: In Python, dynamic protocols are abstract base classes (ABCs) that define a set of methods and attributes that an object must implement to be considered compliant with the protocol. These protocols are typically defined in the <code>collections</code> or <code>typing</code> module.<br>
&emsp;&emsp;Example: The <code>Sequence</code> protocol from the <code>collections</code> module defines a set of methods (<code>__getitem__</code>, <code>__len__</code>, etc.) that an object must implement to be considered a sequence.</p>

<p><b>Static Protocol</b>:<br>
&emsp;&emsp;Definition: Python static protocols are similar to dynamic protocols but make use of the <code>typing.Protocol</code> class from the <code>typing</code> module. These protocols use type hints to define the expected interface (methods and attributes) of a class at static type-checking time. Static protocols provide better tooling support and more explicit type checking.<br>
&emsp;&emsp;Example: A protocol can define that an object must have certain methods and attributes. For instance, a <code>Printable</code> protocol may require an object to implement a <code>print</code> method.</p>

<h2>Monkey Patching as a Means of Implementing a Protocol at Runtime</h2>

Monkey patching is a technique in which the behavior of existing code (classes, methods, functions, etc.) is modified at runtime. This can involve adding, modifying, or replacing existing methods or attributes.

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])


class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [2]:
from random import shuffle

try:
    deck = FrenchDeck()
    shuffle(deck)
except TypeError as e:
    print(e.__repr__())

TypeError("'FrenchDeck' object does not support item assignment")


Monkey patching of the `FrenchDeck` class to make it modifiable and compatible with the `rundom.shuffle` function

In [3]:
def set_card(deck, position, card):
    """
    Function that accepts deck, position, and card arguments.
    """
    deck._cards[position] = card

In [4]:
# Assign a function to the __setitem__ attribute of the FrenchDeck class
FrenchDeck.__setitem__ = set_card

In [5]:
shuffle(deck)
deck[:5]

[Card(rank='3', suit='spades'),
 Card(rank='8', suit='clubs'),
 Card(rank='Q', suit='spades'),
 Card(rank='2', suit='clubs'),
 Card(rank='5', suit='hearts')]

<h2>ABC Subclass Creation</h2>

`abc.MutableSequence` is an abstract base class (ABC). It serves as a blueprint for creating mutable sequence data types, such as lists, that can be modified after creation.<br>
Purpose: Provides a standard set of methods that subclasses must implement to be considered mutable sequences.

In [6]:
from collections import abc, namedtuple

Card = namedtuple('Card', ['rank', 'suit'])


class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init_(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.rank]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):
        self._cards[position] = value

In [7]:
try:
    deck = FrenchDeck2()
except TypeError as e:
    print(e.__repr__())

TypeError("Can't instantiate abstract class FrenchDeck2 without an implementation for abstract methods '__delitem__', 'insert'")


To create a subclass of `MutableSequence`, it is necessary to implement the `__delitem__`, `insert` abstract methods defined in this ABC.

In [8]:
from collections import abc, namedtuple

Card = namedtuple('Card', ['rank', 'suit'])


class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init_(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.rank]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):
        self._cards[position] = value

    def __delitem__(self, position):
        del self._cards[position]

    def insert(self, position, value):
        self._cards.insert(position, value)